Welcome to horsetail matching. This tutorial shows you how to setup and run a very simple horsetail matching optimization. This assumes you have followed the installation tutorial. First we import the module...

In [1]:
from horsetailmatching import ProbabilisticParameter, IntervalParameter, HorsetailMatching
from horsetailmatching.demoproblems import TP1

The first thing required for an optimization under uncertainty is a function to optimize. The horsetail matching python module comes with several test problems, which we will use here. We require a function that takes two arguments: the value of the design variables, and the value of the uncertainties. It should return the value quantity of interest.  

In [2]:
my_function = TP1
print TP1([0,1],[1,0])

2.0


Now we must define the input uncertainties to the problem. These are created as horsetail matching parameters, which have the method evalPDF which allows the horsetail matching object to sample them accordingly. Here we create a uniformly distributed uncertainty and a gaussian uncertainty.

In [3]:
u1 = ProbabilisticParameter(distribution='uniform', lower_bound=-1, upper_bound=1)
u2 = ProbabilisticParameter(distribution='gaussian', mean=0, standard_deviation=1)
input_uncertainties = [u1, u2]

Now we create the HorsetailMatching object which will do the evalation of the horsetail matching metric and its gradient. By default a standard target of t(h) = 0 is used. In this initialization we can also specify details such as how many samples to take to estimate the CDF, which we set to 1000, and the method we want horsetail matching to use to estimate the CDF, which we set to 'empirical' (meaning use the empirical CDF). 

In [4]:
theHM = HorsetailMatching(uparams=input_uncertainties, fqoi=my_function, n_samples_prob=1000, method='empirical')

Then we use this objects methods to evaluate the horsetail matching metric at a given design point. We can also specify how we evaluate the metric using the 'method' argument. Here we use the empirical CDF which we asked for when creating the instance, and compare it to when using gaussian kernels:

In [6]:
ansEmpirical = theHM.evalMetric(x=[1,1])
ansKernel = theHM.evalMetric(x=[1,1], method='kernel')

print('Empirical: {0}, Kernel: {1}, Difference: {2}'.format(ansEmpirical, ansKernel, abs(ansEmpirical-ansKernel)))

Empirical: 1.41421356237, Kernel: 1.41421356237, Difference: 0.0


Now if we want to use this within an optimization, its a simple as passing theHM.evalMetric